In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from funciones_2 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Detección de Fraude en Transacciones de Tarjetas de Crédito
El conjunto de datos de este problema comprende transacciones realizadas mediante tarjetas de crédito en un lapso de dos días, contabilizando 492 casos de fraude entre un total de 284,807 transacciones. Este dataset presenta un desequilibrio considerable, ya que la clase positiva (fraude) representa solo el 0.17 % del total de transacciones. Para una descripción más detallada del dataset, consulte el archivo credit card description.md.
En este problema, nuestro objetivo es desarrollar varios modelos para clasificar una transacción como fraudulenta o no, y luego evaluar la eficacia de cada uno. Para esto, el conjunto de datos se dividió previamente en uno de entrenamiento (credit card train.csv ), uno de validaci ́on (credit card valid.csv) y uno de testeo (credit card test.csv). Estos conjuntos permanecerán fijos durante el desarrollo de los modelos (es decir, en este problema tampoco aplicaremos validación cruzada). En caso de que sea necesario ajustar un hiperparámetro, esto se hará evaluando la métrica de performance sobre el conjunto de validación.

In [7]:
train = pd.read_csv('/Users/maxi/Downloads/Actuales/ML/ML-TP3/Data/2 - Detección de Fraude en Transacciones de Tarjetas de Crédito/credit_card_train.csv')
test = pd.read_csv('/Users/maxi/Downloads/Actuales/ML/ML-TP3/Data/2 - Detección de Fraude en Transacciones de Tarjetas de Crédito/credit_card_test.csv')
validation = pd.read_csv('/Users/maxi/Downloads/Actuales/ML/ML-TP3/Data/2 - Detección de Fraude en Transacciones de Tarjetas de Crédito/credit_card_valid.csv')

# normalize all data
# min_max = get_min_max(train)
# train = min_max_normalize(train, min_max)
# test = min_max_normalize(test, min_max)
# validation = min_max_normalize(validation, min_max)

# if any column has a missing value replace with cero
# train = train.fillna(0)
# test = test.fillna(0)
# validation = validation.fillna(0)

X_train = train.drop(columns=['Class'])
y_train = train['Class'].values

X_test = test.drop(columns=['Class'])
y_test = test['Class'].values

X_validation = validation.drop(columns=['Class'])
y_validation = validation['Class'].values

print("train shape: ", train.shape)
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)

print("\ntest shape: ", test.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

print("\nvalidation shape: ", validation.shape)
print("X_validation shape: ", X_validation.shape)
print("y_validation shape: ", y_validation.shape)

pd.set_option('display.max_columns', None)
train.head()
test.head()
validation.head()
# X_train.head()
# y_train

# # show all rows
# pd.set_option('display.max_rows', None)
# train

train shape:  (182276, 31)
X_train shape:  (182276, 30)
y_train shape:  (182276,)

test shape:  (56962, 31)
X_test shape:  (56962, 30)
y_test shape:  (56962,)

validation shape:  (45569, 31)
X_validation shape:  (45569, 30)
y_validation shape:  (45569,)


,Unnamed: 0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class,Log Amount
0,35473,-0.837642,1.112979,2.028655,3.145547,-0.837502,0.627196,-0.414030,0.742542,-0.701972,0.405014,-0.879133,-0.682846,-1.343436,0.294331,0.621764,-0.500612,0.897365,-0.107476,0.453730,-0.166689,0.268309,0.821847,0.046542,0.397589,-0.479563,0.336809,-0.037172,0.075603,0,3.196671
1,95426,1.316548,0.414713,-0.018404,0.487436,0.132814,-0.574483,0.145913,-0.215796,-0.080218,-0.314228,-0.623766,0.557716,1.498034,-0.561625,1.030891,0.669708,-0.373979,-0.164469,0.109892,0.017367,-0.329864,-0.890929,0.011784,-0.477604,0.365847,0.143306,-0.016664,0.024555,0,0.399447
2,61436,-0.571404,0.258989,1.287805,-1.869622,-0.009301,-0.635431,0.685077,-0.022068,-2.008916,0.031738,1.695940,-0.012262,-0.189118,0.213160,-0.655677,0.850890,0.251749,-1.814555,0.106243,0.097562,0.053786,-0.121669,0.004595,0.188679,-0.022828,-0.603318,0.042462,0.085269,0,3.772324
3,62222,1.257227,0.354853,0.306366,0.693287,-0.372857,-1.073952,0.084993,-0.202787,0.035047,-0.281952,-0.068773,0.226212,0.209235,-0.284310,1.095689,0.479185,-0.004271,-0.297321,-0.198207,-0.082833,-0.288084,-0.830980,0.130011,0.344891,0.215693,0.094511,-0.023087,0.030703,0,0.255417
4,156230,-0.105025,1.309503,-0.365174,-0.391590,1.033099,-0.811986,0.874705,-0.203432,1.362339,-0.905523,0.338341,-2.368896,2.124732,0.252604,-1.009552,0.265979,1.054074,-0.008186,-0.455076,0.139322,-0.517254,-1.010887,0.116913,0.507027,-0.370831,0.078228,0.300587,0.148577,0,0.989913


## (a)
Implementar los siguientes clasificadores sobre el conjunto de datos de entrenamiento sin aplicar ninguna técnica de re-balanceo, y para cada uno reportar la matriz de confusión, accuracy, precision, recall, curva ROC y área bajo la curva ROC (AUC-ROC), curva PRC (Precision-Recall Curve) y  ́area bajo la curva PRC (AU-PRC) sobre el conjunto de validación:

 - Red neuronal densamente conectada con una capa oculta de 16 nodos y función de activación sigmoide. Entrenar durante 150 epochs utilizando el optimizador ADAM con un learning rate de 0.001 y un batch size de 2048, y la entrop ́ıa cruzada binaria como función de costo. Si quiere explorar diferentes configuraciones, y quedarse con la mejor, lo puede hacer.
Opcional: Implementar la técnica de “Early Stopping” monitoreando AU-PRC (área bajo la curva Precision-Recall) con una paciencia de 10 epochs.

 - Bosque aleatorio (Random Forest) con 20 árboles, utilizando la entropía como criterio de división, estableciendo una profundidad máxima de 10 para cada árbol. Si quiere explorar diferentes configuraciones, y quedarse con la mejor, lo puede hacer.

### Red Neuronal 

In [8]:
# fit MLP model
X_train.shape[1]
mlp = MLP(X_train.shape[1], [30, 16, 1], ['sigmoid', 'sigmoid', 'sigmoid'])

train_losses_1, test_losses_1 = mlp.fit(list(zip(X_train.values, y_train)), 
                                        list(zip(X_validation.values, y_validation)), 
                                        48, 
                                        alpha=0.001, 
                                        max_epochs=150)



  0%|          | 0/150 [00:00<?, ?it/s]/Users/maxi/Downloads/Actuales/ML/ML-TP3/Problema 2/funciones_2.py:147: RuntimeWarning: overflow encountered in exp
  return lambda z : 1 / (1 + np.exp(-z))
/Users/maxi/Downloads/Actuales/ML/ML-TP3/Problema 2/funciones_2.py:166: RuntimeWarning: overflow encountered in multiply
  return lambda z : z * (1 - z)
/Users/maxi/Downloads/Actuales/ML/ML-TP3/Problema 2/funciones_2.py:261: RuntimeWarning: invalid value encountered in add
  nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, d_nabla_w)]
  0%|          | 0/150 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [13]:
print(f"num layers: {mlp.num_layers}")
print(f"layers: {mlp.layers}")
print(f"activations: {mlp.activations}")

num layers: 3
layers: [30, 16, 1]
activations: ['sigmoid', 'sigmoid', 'sigmoid']


In [5]:
y_mlp_valid_pred = mlp.predict(X_validation.values)

NameError: name 'mlp' is not defined

In [6]:
TP, TN, FP, FN = confusion_matrix(y_validation, y_mlp_valid_pred)

In [7]:
print(f"Confusion Matrix: \n{np.array([[TN, FP], [FN, TP]])}")

Confusion Matrix: 
[[0 0]
 [0 0]]


### Random Forest

In [11]:
dec_tree = DecisionTree(max_depth=3)
dec_tree.fit(X_train.values, y_train)
y_dec_tree_valid_pred = dec_tree.predict(X_validation)

  0%|          | 0/30 [00:09<?, ?it/s]


KeyboardInterrupt: 

## (b)
Volver a entrenar ambos modelos (red neuronal y bosque aleatorio), aplicando cada una de las siguientes técnicas de re-balanceo. Para cada modelo, y cada tecnica de re-balanceo, reportar las métricas de performance mencionadas en el inciso anterior.

 - Undersampling: eliminar muestras de la clase mayoritaria de manera aleatoria hasta que ambas clases tengan igual proporción.

 - Oversampling by duplication: duplicar muestras de la clase minoritaria de manera aleatoria, hasta que que ambas clases tengan igual proporción.

 - Cost re-weighting: en la función de costo, multiplicar los terminos que dependen de las muestras de la clase minoritaria por un factor $C = \frac{\pi _2} {\pi _1}$
 - SMOTE (Synthetic Minority Oversampling Technique): hasta que ambas clases tengan igual proporción.
 - Opcional: Explorar distintas configuraciones de red neuronal (diferente nu ́mero de capas, unidades ocultas, optimizador, learning rate, batch size) y/o del bosque aleatorio y reportar cual cree que es “el mejor”.

## (c)
Sea un problema de clasifiación binaria, donde el proceso estocástico que genera los datos tiene una distribución a posteriori $P(Cj/x)$ tal que para N suficientemente grande las clases no son linealmente separables en el espacio de features (es decir, las muestras de las dos clases tienen algun grado de solapamiento cuando son proyectadas sobre el espacio de features). Sean $π1 = P(C1)$ y $π2 = P(C2)$ las probabilidades marginales de que el proceso genere una muestra de la clase 1 y la clase 2, respectivamente, explicar por qué si utilizamos regresión logística binaria, sin aplicar ninguna técnica de re-balanceo, a medida que $π1$ tiende a 0 y con $N$ suficientemente grande, el accuracy tiende a 1 mientras que el precision tiende a 0.

NOTA: Este resultado es general, y aplica a cualquier clasificador “no-sezgado” (como los que vimos en clase), no solamente a regresión log ́ıstica.
